# DEFINITIONS

In [167]:
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim

In [2]:
# Get proper training data
def get_xy(filename, max_frames, max_classes):

    data = json.load(open(filename, 'r'))
    xy_data = []
    classes = {"10":0, "00":0, "11":0, "01":0}

    for entry in data.items():
        entry_content = entry[1]
        entry_val = int(entry_content["valence"])
        entry_act = int(entry_content["activation"])
        entry_emotion = f"{entry_val}{entry_act}"
        entry_frames = entry_content["features"]

        # Normalize frames
        for frame in entry_frames:
            for counter, number in enumerate(frame):
                frame[counter] = float(round(number, 5))

        # Equalize classes
        for key in classes:
            if entry_emotion == key:
                if classes[key] < max_classes:
                    classes[key] += 1
                    xy_data.append([entry_frames,float(entry_val),float(entry_act)])

    # Equalize frames
    for entry in xy_data:
        features_length = len(entry[0])
        difference = features_length - max_frames
        padding = [float(0) for counter in range(26)]
        if difference < 0:
            # Add the missing frames, each frame with 26 zeroes
            for counter in range(abs(difference)):
                entry[0].append(padding)
        else:
            for counter in range(difference):
                entry[0].pop()

    return xy_data

class DL_Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        entry = self.data[index]
        # Have to add another dimension because using CNN in Pytorch is dumb
        frames = torch.tensor((entry[0])).unsqueeze(0)
        val_label = torch.tensor((entry[1])).unsqueeze(0)
        act_label = torch.tensor((entry[2])).unsqueeze(0)
        return frames, val_label, act_label

# Get proper testing data
def get_sxy(filename, max_frames):

    data = json.load(open(filename, 'r'))
    xy_data = []

    for entry in data.items():
        entry_content = entry[1]
        entry_frames = entry_content["features"]

        # Normalize frames
        for frame in entry_frames:
            for counter, number in enumerate(frame):
                frame[counter] = float(round(number, 5))

        xy_data.append([entry_frames])

    # Equalize frames
    for entry in xy_data:
        features_length = len(entry[0])
        difference = features_length - max_frames
        padding = [float(0) for counter in range(26)]
        if difference < 0:
            # Add the missing frames, each frame with 26 zeroes
            for counter in range(abs(difference)):
                entry[0].append(padding)
        else:
            for counter in range(difference):
                entry[0].pop()

    return xy_data    

class DL_SDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        entry = self.data[index]
        frames = torch.tensor((entry[0])).unsqueeze(0)
        return frames

# DATA HANDLING

In [3]:
max_frames = 200
max_classes = 1000
batch_size = 100

In [4]:
train_data = DL_Dataset(get_xy('train.json', max_frames, max_classes))
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = False)

In [5]:
test_data = DL_Dataset(get_xy('dev.json', max_frames, max_classes))
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = False)

In [299]:
sub_data = DL_SDataset(get_sxy('ser_test_2.json', max_frames))
sub_loader = torch.utils.data.DataLoader(dataset = sub_data, batch_size = batch_size, shuffle = False)

# MODEL

In [332]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()

        self.v1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=2),
            nn.ReLU(), nn.BatchNorm2d(16), nn.MaxPool2d(2, 2))
        self.v2 = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=2),
            nn.ReLU(), nn.BatchNorm2d(16), nn.MaxPool2d(kernel_size=2, stride=2))
        self.v3 = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=2),
            nn.ReLU(), nn.BatchNorm2d(16), nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten())
        self.v4 = nn.Sequential(nn.Linear(2080, 1), nn.Sigmoid())
        
        self.a1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=2),
            nn.ReLU(), nn.BatchNorm2d(16), nn.MaxPool2d(2, 2))
        self.a2 = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=2),
            nn.ReLU(), nn.BatchNorm2d(16), nn.MaxPool2d(kernel_size=2, stride=2))
        self.a3 = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=2),
            nn.ReLU(), nn.BatchNorm2d(16), nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten())
        self.a4 = nn.Sequential(nn.Linear(2080, 1), nn.Sigmoid())

    def forward(self, x):
        vout = self.v1(x)
        vout = self.v2(vout)
        vout = self.v3(vout)
        vout = self.v4(vout)

        aout = self.a4(self.a3(self.a2(self.a1(x))))

        return vout, aout

device = torch.device('cpu')
model = Model2().to(device)
num_epochs = 5
learning_rate = 0.0001
loss_fn1 = nn.BCELoss()
loss_fn2 = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# TRAINING

In [ ]:
for epoch in range(num_epochs):
    cc = 0
    for frames, val, act in train_loader:
        frames, val, act = frames.to(device), val.to(device), act.to(device)
        optimizer.zero_grad()

        vout, aout = model(frames)
        loss_val = loss_fn1(vout, val)
        loss_act = loss_fn2(aout, act)
        total_loss = loss_val + loss_act

        cc += 1
        if cc == 1:
            #print(f"{frames.size()} {val.size()} {act.size()} {vout.size()} {aout.size()}")
            pass

        loss_val.backward()
        loss_act.backward()
        optimizer.step()
        
        #print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}, VL: {loss_val}, AL: {loss_act}')

        testo = {"v1":0, "v0":0, "a1":0, "a0":0}
        for i in range(len(vout)):
            if vout[i] > 0.5:
                testo["v1"] += 1
            else:
                testo["v0"] += 1
        for i in range(len(aout)):
            if aout[i] > 0.5:
                testo["a1"] += 1
            else:
                testo["a0"] += 1                 
        #print(testo)

# Evaluation

In [ ]:
model.eval()

In [296]:
vcor = 0
acor = 0
entries = 0 # 30 batches x 100 entries
for frames, val, act in test_loader:
    entries += val.size(0)

    vout, aout = model(frames)

    listy = [val, act, vout, aout]
    listy2 = []
    for i in listy:
        listy3 = []
        for y in i:
            if y.item() >= 0.5:
                listy3.append(1)
            else:
                listy3.append(0)
        listy2.append(listy3)
    
    for i in range(len(val)):
        if listy2[0][i] == listy2[2][i]:
            vcor += 1
        if listy2[1][i] == listy2[3][i]:
            acor += 1

    if entries == 100:
        #!!!!
        #print(act[0].item())
        #print(aout[0].item())
        #print(f"{vout.size()} {val.size()} {act.size()} {vout.size()} {aout.size()}")
        pass

#print(f"entries = {entries} / vacc = {(vcor/entries)} / aacc = {(acor/entries)} / vcor: {vcor} / acor: {acor} / loss: {total_loss.item()}")

entries = 2928 / vacc = 0.4959016393442623 / aacc = 0.5836748633879781 / vcor: 1452 / acor: 1709 / loss: 0.35127782821655273


# SUBMISSION

In [335]:
val_list = []
act_list = []
for frames in sub_loader:
    vout, aout = model(frames)
    for i in vout:
        for y in i:
            if y.item() >= 0.5:
                val_list.append(1)
            else:
                val_list.append(0)
    for i in aout:
        for y in i:
            if y.item() >= 0.5:
                act_list.append(1)
            else:
                act_list.append(0)

final = {}
for i in range(len(val_list)):
    final[str(i)] = {"valence": val_list[i], "activation": act_list[i]}

with open('submission.json', 'w') as f:
    json.dump(final, f)

In [336]:
testo1 = {1:0, 0:0}
testo2 = {1:0, 0:0}
for i in val_list:
    if i == 0:
        testo1[0] += 1
    elif i == 1:
        testo1[1] += 1
    else:
        print(i)
for i in act_list:
    if i == 0:
        testo2[0] += 1
    elif i == 1:
        testo2[1] += 1
    else:
        print(i)
print(testo1)
print(testo2)

{1: 5587, 0: 851}
{1: 3273, 0: 3165}
